In [2]:
import os
import time
import requests
import json
import undetected_chromedriver as uc
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image
import pytesseract

# Set the working directory and environment variables
os.chdir('/allah/data')
os.environ['DISPLAY'] = ':10.0'  # Adjust if necessary


# Configure Chrome options
options = Options()
# options.add_argument("start-maximized")  # Open in maximized mode
options.add_argument('--disable-blink-features=AutomationControlled')  # Disable automation control features
options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems
options.add_argument('--no-sandbox')  # Bypass OS security model
options.add_argument('--disable-gpu')  # Disable GPU hardware acceleration
options.binary_location = "/usr/bin/google-chrome" 

driver = uc.Chrome(options=options)

highlighted_elements = []

def highlight(element):
    driver.execute_script("arguments[0].style.border='3px solid red'", element)
    highlighted_elements.append(element)

def clear_all_highlights():
    for element in highlighted_elements:
        driver.execute_script("arguments[0].style.border=''", element)
    highlighted_elements.clear()



In [ ]:
driver.get('https://bclub.tk/login/')

# Fill in username and password
username_input = driver.find_element(By.ID, 'id_username')
highlight(username_input)
username_input.send_keys('Brian9977km1')

password_input = driver.find_element(By.ID, 'password')
highlight(password_input)
password_input.send_keys('nimasile')

In [1]:
import os
import time
import requests
import json
import undetected_chromedriver as uc
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm.notebook import tqdm

# Set the working directory and environment variables
os.chdir('/allah/data')
os.environ['DISPLAY'] = ':10.0'  # Adjust if necessary

# Configure Chrome options
options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--no-sandbox')
options.add_argument('--disable-gpu')
options.binary_location = "/usr/bin/google-chrome"

# Initialize Chrome driver
driver = uc.Chrome(options=options)

# Define the base URL and target data path
base_url = 'https://data.binance.vision/?prefix=data/futures/um/daily/'
data_path = '/allah/data'
ticker = 'ETHUSDT'

# Function to highlight elements (not used in final code but retained for completeness)
highlighted_elements = []
def highlight(element):
    driver.execute_script("arguments[0].style.border='3px solid red'", element)
    highlighted_elements.append(element)

def clear_all_highlights():
    for element in highlighted_elements:
        driver.execute_script("arguments[0].style.border=''", element)
    highlighted_elements.clear()

# Navigate to the page and wait for elements to load
driver.get(base_url)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#listing > tr:nth-child(2) > td:nth-child(1)')))

# Extract links and categories
tbody_element = driver.find_element(By.ID, 'listing')
anchor_elements = tbody_element.find_elements(By.TAG_NAME, 'a')
links = [(anchor.get_attribute('href'), anchor.text.strip('/')) for anchor in anchor_elements][1:]

# Create directories for each category and collect category names
categories = [link[1] for link in links]
for category in categories:
    os.makedirs(os.path.join(data_path, ticker, category), exist_ok=True)

# Function to download a file with retries
def download_file(url, download_path):
    response = requests.get(url, stream=True)
    with open(download_path, 'wb') as file:
        for data in response.iter_content(1024):
            file.write(data)

# Function to process a category
def process_category(category, ticker, base_url):
    url = f'{base_url}{category}/{ticker}/'
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#listing > tr:nth-child(2) > td:nth-child(1)')))
    
    tbody_element = driver.find_element(By.ID, 'listing')
    download_links = [(a.get_attribute('href'), a.text.strip('/')) for a in tbody_element.find_elements(By.TAG_NAME, 'a') if a.text.endswith('.zip')]
    
    total_files = len(download_links)
    progress_bar = tqdm(total=total_files, unit='file', desc=f'Downloading {category}', colour='green', leave=True, ncols=100, dynamic_ncols=True, position=0)
    
    for link, filename in download_links:
        download_path = os.path.join(data_path, ticker, category, filename)
        os.makedirs(os.path.dirname(download_path), exist_ok=True)
        if os.path.exists(download_path):
            print(f'Skipped {filename} (already exists)')
            progress_bar.update(1)
            continue
        
        retry_count = 0
        while retry_count < 5:
            try:
                download_file(link, download_path)
                progress_bar.update(1)
                break
            except requests.exceptions.ConnectionError as e:
                retry_count += 1
                print(f'Error downloading {filename}: {e}. Retrying ({retry_count}/5)...')
                time.sleep(20)
        else:
            print(f'Failed to download {filename} after 5 attempts')
    
    progress_bar.close()

# Function to process Kline categories
def process_kline_category(kline_category, ticker, base_url):
    category_url = f'{base_url}{kline_category}/{ticker}/'
    driver.get(category_url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#listing > tr:nth-child(2) > td:nth-child(1)')))
    
    tbody_element = driver.find_element(By.ID, 'listing')
    time_frame_links = [(a.get_attribute('href'), a.text.strip('/')) for a in tbody_element.find_elements(By.TAG_NAME, 'a')][1:]
    
    for time_link, time_frame in time_frame_links:
        driver.get(time_link)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#listing > tr:nth-child(2) > td:nth-child(1)')))
        
        tbody_element = driver.find_element(By.ID, 'listing')
        download_links = [(a.get_attribute('href'), a.text.strip('/')) for a in tbody_element.find_elements(By.TAG_NAME, 'a') if a.text.endswith('.zip')]
        
        total_files = len(download_links)
        progress_bar = tqdm(total=total_files, unit='file', desc=f'Downloading {kline_category} - {time_frame}', colour='green', leave=True, ncols=100, dynamic_ncols=True, position=0)
        
        for link, filename in download_links:
            download_path = os.path.join(data_path, ticker, kline_category, time_frame, filename)
            os.makedirs(os.path.dirname(download_path), exist_ok=True)
            if os.path.exists(download_path):
                print(f'Skipped {filename} (already exists)')
                progress_bar.update(1)
                continue
            
            retry_count = 0
            while retry_count < 5:
                try:
                    download_file(link, download_path)
                    progress_bar.update(1)
                    break
                except requests.exceptions.ConnectionError as e:
                    retry_count += 1
                    print(f'Error downloading {filename}: {e}. Retrying ({retry_count}/5)...')
                    time.sleep(20)
            else:
                print(f'Failed to download {filename} after 5 attempts')
        
        progress_bar.close()

# Extract categories and identify Kline categories
kline_categories = [category for category in categories if 'klines' in category.lower()]

# Process all categories
for category in categories:
    if category in kline_categories:
        process_kline_category(category, ticker, base_url)
    else:
        process_category(category, ticker, base_url)


In [4]:
import os
import zipfile
from tqdm.notebook import tqdm

def unzip_files(base_directory, target_base_directory):
    # Collect all zip files
    zip_files = []
    for root, dirs, files in os.walk(base_directory):
        for file in files:
            if file.endswith(".zip"):
                zip_files.append(os.path.join(root, file))
    
    # Iterate over the zip files with a progress bar
    for file_path in tqdm(zip_files, desc="Unzipping files"):
        relative_path = os.path.relpath(os.path.dirname(file_path), base_directory)
        unzip_dir = os.path.join(target_base_directory, relative_path)
        os.makedirs(unzip_dir, exist_ok=True)  # Create the unzip directory if it doesn't exist
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)
        print(f'Unzipped: {file_path} to {unzip_dir}')

# Define the directory containing the zip files
base_directory = '/allah/data/ETHUSDT/trades'

# Define the target base directory where files will be unzipped
target_base_directory = '/mnt/data'

# Call the function to unzip files
unzip_files(base_directory, target_base_directory)


Unzipping files:   0%|          | 0/1698 [00:00<?, ?it/s]

Unzipped: /allah/data/ETHUSDT/trades/ETHUSDT-trades-2022-10-26.zip to /mnt/data/.
Unzipped: /allah/data/ETHUSDT/trades/ETHUSDT-trades-2021-12-26.zip to /mnt/data/.
Unzipped: /allah/data/ETHUSDT/trades/ETHUSDT-trades-2020-02-27.zip to /mnt/data/.
Unzipped: /allah/data/ETHUSDT/trades/ETHUSDT-trades-2022-10-23.zip to /mnt/data/.
Unzipped: /allah/data/ETHUSDT/trades/ETHUSDT-trades-2020-03-20.zip to /mnt/data/.
Unzipped: /allah/data/ETHUSDT/trades/ETHUSDT-trades-2021-10-11.zip to /mnt/data/.
Unzipped: /allah/data/ETHUSDT/trades/ETHUSDT-trades-2021-03-27.zip to /mnt/data/.
Unzipped: /allah/data/ETHUSDT/trades/ETHUSDT-trades-2019-12-08.zip to /mnt/data/.
Unzipped: /allah/data/ETHUSDT/trades/ETHUSDT-trades-2020-09-19.zip to /mnt/data/.
Unzipped: /allah/data/ETHUSDT/trades/ETHUSDT-trades-2023-12-04.zip to /mnt/data/.
Unzipped: /allah/data/ETHUSDT/trades/ETHUSDT-trades-2020-07-19.zip to /mnt/data/.
Unzipped: /allah/data/ETHUSDT/trades/ETHUSDT-trades-2021-10-08.zip to /mnt/data/.
Unzipped: /allah

KeyboardInterrupt: 